# Tokenizer

- Normalization
- Pre-tokenization
- Build models
- Training the tokenizer

## Pre-tokenizer

In [49]:
operators3 = {'<<=', '>>='}
operators2 = {
    '->', '++', '--',
    '!~', '<<', '>>', '<=', '>=',
    '==', '!=', '&&', '||', '+=',
    '-=', '*=', '/=', '%=', '&=', '^=', '|='
}
operators1 = {
    '(', ')', '[', ']', '.',
    '+', '-', '*', '&', '/',
    '%', '<', '>', '^', '|',
    '=', ',', '?', ':', ';',
    '{', '}'
}
def tokenize(line):
        line = line.replace("\n"," ").replace("\r\n"," ").replace("\r"," ")
        tmp, w = [], []
        i = 0
        while i < len(line):
            # Ignore spaces and combine previously collected chars to form words
            if line[i] == ' ':
                tmp.append(''.join(w))
                tmp.append(line[i])
                w = []
                i += 1
            # Check operators and append to final list
            elif line[i:i + 3] in operators3:
                tmp.append(''.join(w))
                tmp.append(line[i:i + 3])
                w = []
                i += 3
            elif line[i:i + 2] in operators2:
                tmp.append(''.join(w))
                tmp.append(line[i:i + 2])
                w = []
                i += 2
            elif line[i] in operators1:
                tmp.append(''.join(w))
                tmp.append(line[i])
                w = []
                i += 1
            # Character appended to word list
            else:
                w.append(line[i])
                i += 1
        # Filter out irrelevant strings
        res = list(filter(lambda c: c != '', tmp))
        return ' '.join(list(filter(lambda c: c != ' ', res)))

In [50]:
def remove_comments(string):
    pattern = r"(\".*?\"|\'.*?\')|(/\*.*?\*/|//[^\r\n]*$)"
    # first group captures quoted strings (double or single)
    # second group captures comments (//single-line or /* multi-line */)
    regex = re.compile(pattern, re.MULTILINE|re.DOTALL)
    def _replacer(match):
        # if the 2nd group is not None, then we have captured a real comment string.
        if match.group(2) is not None:
            return "" 
        else: # otherwise, we will return the 1st group
            return match.group(1) 
    return regex.sub(_replacer, string)

In [51]:
operators  = ['++', '--', '-', '!', '~', '**', '*', '/', '%', '+','<<', '>>', '&', '^', '|','<', '>', '<=', '>=', '==', '!=' , '&&', '||', '=', '|=', '^=', '&=', '<<=', '>>=', '+=', '-=', '*=', '/=', '%=', ',', ':'  ]
global_variable = ['abi.decode', 'abi.encode', 'abi.encodePacked', 'abi.encodeWithSelector', 'abi.encodeCall', 'abi.encodeWithSignature','bytes.concat','string.concat', 'block.basefee', 'block.chainid',  'block.coinbase', 'block.difficulty', 'block.gaslimit', 'block.number', 'block.prevrandao',  'block.timestamp', 'gasleft', 'msg.data','msg.sender','msg.sig','msg.value', 'tx.gasprice', 'tx.origin' , 'assert', 'require' , 'revert', 'blockhash' , 'keccak256', 'sha256' , 'ripemd160', 'ecrecover', 'addmod', 'mulmod', 'selfdestruct' ]
Other_keyword = ['bytes1','bytes2','bytes3','bytes4','bytes5','bytes6','bytes7','bytes8','bytes9','bytes10','bytes11','bytes12','bytes13','bytes14','bytes15','bytes16','bytes17','bytes18','bytes19','bytes20','bytes21','bytes22','bytes23','bytes24','bytes25','bytes26','bytes27','bytes28','bytes29','bytes30','bytes31','bytes32']
number = ['wei','gwei','ether','seconds','minutes','hours','days','weeks','years' ]
evm_built = ['stop','add','sub','mul','div','sdiv','mod','smod','exp','not','lt','gt','slt','sgt','eq','iszero','and','or','xor','byte','shl','shr','sar','addmod','mulmod','signextend','keccak256','pop','mload','mstore','mstore8','sload','sstore','msize','gas','address','balance','selfbalance','caller','callvalue','calldataload','calldatasize','calldatacopy','extcodesize','extcodecopy','returndatasize','returndatacopy','extcodehash','create','create2','call','callcode','delegatecall','staticcall','return','revert','selfdestruct','invalid','log0','log1','log2','log3','log4','chainid','origin','gasprice','blockhash','coinbase','timestamp','number','difficulty','prevrandao','gaslimit','basefee']
uint = ['uint','uint8','uint16','uint24','uint32','uint40','uint48','uint56','uint64','uint72','uint80','uint88','uint96','uint104','uint112','uint120','uint128','uint136','uint144','uint152','uint160','uint168','uint176','uint184','uint192','uint200','uint208','uint216','uint224','uint232','uint240','uint248','uint256']
int_ =['int','int8','int16','int24','int32','int40','int48','int56','int64','int72','int80','int88','int96','int104','int112','int120','int128','int136','int144','int152','int160','int168','int176','int184','int192','int200','int208','int216','int224','int232','int240','int248','int256']

In [52]:
from datasets import Dataset
import pandas as pd
import re



In [53]:
data = pd.read_pickle("/home/bombbom/Documents/NLP_in_Detection_System/dataset_example/labeled_SBW_datasets.pkl")


In [54]:
data

,address,source_code,categories,lines
0,0x21e13cb3f3f26f92a62ac7adab4093e8997d1fb1,pragma solidity ^0.4.18;\n\n// ---------------...,"['Other', 'arithmetic', 'arithmetic']","[154, 141, 103]"
1,0x344005c29af957567f0b40950b425ed018b92170,pragma solidity ^0.4.21;\n\n\ncontract DSMath...,"['arithmetic', 'reentrancy', 'arithmetic', 'ar...","[201, 16, 209, 176, 146, 154, 29]"
2,0x44e320110176c11c93e116f6770f13d96deded43,pragma solidity ^0.4.20;\n\n\ncontract Ownable...,['arithmetic'],"[82, 46]"
3,0x5ecd84482176db90bb741ddc8c2f9ccc290e29ce,pragma solidity ^0.4.8;\ncontract Token{\n ...,"['arithmetic', 'unchecked_low_calls', 'arithme...","[99, 38, 78, 48, 49, 60, 93]"
4,0xcaf187eb618d2335b4130d784a697be96f4b07b9,pragma solidity 0.4.15;\n\ncontract RegistryIC...,['Other'],"[80, 28, 63]"
...,...,...,...,...
47326,0x7f650f3b231d3a32c2b0e2940e870acdd4aa9961,pragma solidity ^0.4.24;\n\n\n\n\n/**\n * @tit...,"['reentrancy', 'arithmetic', 'time_manipulatio...","[258, 264, 245, 253, 287]"
47327,0xc73f2474001ad1d6aed615af53631148cf98de6b,pragma solidity ^0.4.18;\n\n// ---------------...,"['Other', 'arithmetic', 'arithmetic']","[200, 218, 128, 102]"
47328,0xb2105f178abe620ddbb86a70afa94bfa9daa4d01,pragma solidity ^0.4.23;\n\nlibrary SafeMath {...,"['arithmetic', 'arithmetic', 'front_running']","[124, 20, 205]"
47329,0xe2d4b960d0c639633582cddef57528461e62083d,pragma solidity ^0.4.23;\n\n\n/// @title Multi...,"['arithmetic', 'Other', 'unchecked_low_calls',...","[140, 142, 23, 282, 410, 157, 287, 296, 302, 3..."


In [55]:
data = data[['source_code']]

In [56]:
data.source_code = data.source_code.apply(remove_comments)

/tmp/ipykernel_4931/3853744767.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.source_code = data.source_code.apply(remove_comments)


In [57]:
data.source_code = data.source_code.apply(tokenize)

/tmp/ipykernel_4931/1830387800.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.source_code = data.source_code.apply(tokenize)


In [58]:
data = Dataset.from_pandas(data)

In [59]:
data

Dataset({
    features: ['source_code'],
    num_rows: 47331
})

In [ ]:
# Using a Python generator, we can avoid Python loading anything into memory until it’s actually necessary. To create such a generator, you just to need to replace the brackets with parentheses:
# training_corpus = (
#     data[i : i + 1000]["source_code"]
#     for i in range(0, len(data), 1000)
# )
# This line of code doesn’t fetch any elements of the dataset; it just creates an object you can use in a Python for loop. The texts will only be loaded when you need them (that is, when you’re at the step of the for loop that requires them), and only 1,000 texts at a time will be loaded. This way you won’t exhaust all your memory even if you are processing a huge dataset.


In [60]:
def get_training_corpus():
    return (
        data[i : i + 1000]["source_code"]
        for i in range(0, len(data), 1000)
    )


training_corpus = get_training_corpus()

In [61]:
training_corpus

<generator object get_training_corpus.<locals>.<genexpr> at 0x7f2e7e460040>

In [62]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [63]:
example = '''pragma solidity ^0.8.17;

contract HelloWorld {
    string public greet = "Hello World!";
}'''

tokens = old_tokenizer.tokenize(example)
tokens

['pr',
 '##ag',
 '##ma',
 'solid',
 '##ity',
 '^',
 '0',
 '.',
 '8',
 '.',
 '17',
 ';',
 'contract',
 'hello',
 '##world',
 '{',
 'string',
 'public',
 'greet',
 '=',
 '"',
 'hello',
 'world',
 '!',
 '"',
 ';',
 '}']

In [64]:
old_tokenizer.add_tokens(list(operators))
old_tokenizer.add_tokens(list(global_variable))
old_tokenizer.add_tokens(list(Other_keyword))
old_tokenizer.add_tokens(list(number))
old_tokenizer.add_tokens(list(evm_built))
old_tokenizer.add_tokens(list(uint))
old_tokenizer.add_tokens(list(int_))

32

In [65]:
old_tokenizer(example).tokens()

['[CLS]',
 'pr',
 '##ag',
 '##ma',
 'solid',
 '##ity',
 '^',
 '0',
 '.',
 '8',
 '.',
 '17',
 ';',
 'contract',
 'hello',
 '##world',
 '{',
 'string',
 'public',
 'greet',
 '=',
 '"',
 'hello',
 'world',
 '!',
 '"',
 ';',
 '}',
 '[SEP]']

In [66]:
old_tokenizer(example)

{'input_ids': [101, 10975, 8490, 2863, 5024, 3012, 1034, 1014, 1012, 1022, 1012, 2459, 1025, 3206, 7592, 11108, 1063, 5164, 2270, 17021, 1027, 1000, 7592, 2088, 999, 1000, 1025, 1065, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [67]:
old_tokenizer.decode(10975)

'pr'

In [68]:
old_tokenizer.is_fast

True

In [69]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)


In [70]:
tokens = tokenizer.tokenize(example)
tokens

['pragma',
 'solidity',
 '^',
 '0',
 '.',
 '8',
 '.',
 '17',
 ';',
 'contract',
 'hello',
 '##world',
 '{',
 'string',
 'public',
 'gre',
 '##et',
 '=',
 '"',
 'hello',
 'world',
 '!',
 '"',
 ';',
 '}']

In [71]:
tokenizer.save_pretrained("/home/bombbom/Documents/NLP_in_Detection_System/save_model/tokenizer/")

('/home/bombbom/Documents/NLP_in_Detection_System/save_model/tokenizer/tokenizer_config.json',
 '/home/bombbom/Documents/NLP_in_Detection_System/save_model/tokenizer/special_tokens_map.json',
 '/home/bombbom/Documents/NLP_in_Detection_System/save_model/tokenizer/vocab.txt',
 '/home/bombbom/Documents/NLP_in_Detection_System/save_model/tokenizer/added_tokens.json',
 '/home/bombbom/Documents/NLP_in_Detection_System/save_model/tokenizer/tokenizer.json')

In [ ]:
# Replace "huggingface-course" below with your actual namespace to use your own tokenizer
tokenizer = AutoTokenizer.from_pretrained("/home/bombbom/Documents/NLP_in_Detection_System/save_model/tokenizer/")